In [ ]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train / 255., x_test / 255.

def plot(samples):
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(8, 8)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        plt.imshow(sample.reshape(28, 28))

    return fig

num_epoch = 100000
batch_size = 64
num_input = 28 * 28
num_latent_variable = 100   # 잠재 변수 z의 차원
num_hidden = 128
learning_rate = 0.001

train_data = tf.data.Dataset.from_tensor_slices(x_train)
train_data = train_data.repeat().shuffle(60000).batch(batch_size)
train_data_iter = iter(train_data)

class Generator(object):
    def __init__(self):
        self.G_W1 = tf.Variable(tf.random.normal(shape=[num_latent_variable, num_hidden], stddev=5e-2))
        self.G_b1 = tf.Variable(tf.constant(0.1, shape=[num_hidden]))
        self.G_W2 = tf.Variable(tf.random.normal(shape=[num_hidden, num_input], stddev=5e-2))
        self.G_b2 = tf.Variable(tf.constant(0.1, shape=[num_input]))

    def __call__(self, X):
        hidden_layer = tf.nn.relu((tf.matmul(X, self.G_W1) + self.G_b1))
        output_layer = tf.matmul(hidden_layer, self.G_W2) + self.G_b2
        generated_mnist_image = tf.nn.sigmoid(output_layer)

        return generated_mnist_image

class Discriminator(object):
    def __init__(self):
        self.D_W1 = tf.Variable(tf.random.normal(shape=[num_input, num_hidden], stddev=5e-2))
        self.D_b1 = tf.Variable(tf.constant(0.1, shape=[num_hidden]))
        self.D_W2 = tf.Variable(tf.random.normal(shape=[num_hidden, 1], stddev=5e-2))
        self.D_b2 = tf.Variable(tf.constant(0.1, shape=[1]))

    def __call__(self, X):
        hidden_layer = tf.nn.relu((tf.matmul(X, self.D_W1) + self.D_b1))
        logits = tf.matmul(hidden_layer, self.D_W2) + self.D_b2
        predicted_value = tf.nn.sigmoid(logits)

        return predicted_value, logits

@tf.function
def generator_loss(D_fake_logits):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake_logits)))         # log(D(G(z))

@tf.function
def discriminator_loss(D_real_logits, D_fake_logits):
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real_logits)))  # log(D(x))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake_logits)))  # log(1-D(G(z)))
    d_loss = d_loss_real + d_loss_fake  # log(D(x)) + log(1-D(G(z)))

    return d_loss

Generator_model = Generator()

Discriminator_model = Discriminator()

discriminator_optimizer = tf.optimizers.Adam(learning_rate)
generator_optimizer = tf.optimizers.Adam(learning_rate)

@tf.function
def d_train_step(discriminator_model, real_image, fake_image):
    with tf.GradientTape() as disc_tape:
        D_real, D_real_logits = discriminator_model(real_image)  # D(x)
        D_fake, D_fake_logits = discriminator_model(fake_image)  # D(G(z))
        loss = discriminator_loss(D_real_logits, D_fake_logits)
    gradients = disc_tape.gradient(loss, vars(discriminator_model).values())
    discriminator_optimizer.apply_gradients(zip(gradients, vars(discriminator_model).values()))

@tf.function
def g_train_step(generator_model, discriminator_model, z):
    with tf.GradientTape() as gen_tape:
        G = generator_model(z)
        D_fake, D_fake_logits = discriminator_model(G)  # D(G(z))
        loss = generator_loss(D_fake_logits)
    gradients = gen_tape.gradient(loss, vars(generator_model).values())
    generator_optimizer.apply_gradients(zip(gradients, vars(generator_model).values()))



if not os.path.exists('generated_output/'):
    os.makedirs('generated_output/')

for i in range(num_epoch):
    batch_X = next(train_data_iter)

    batch_noise = np.random.uniform(-1., 1., [batch_size, 100]).astype('float32')

    if i % 500 == 0:
        samples = Generator_model(np.random.uniform(-1., 1., [64, 100]).astype('float32')).numpy()
        fig = plot(samples)
        plt.savefig('generated_output/%s.png' % str(num_img).zfill(3), bbox_inches='tight')
        num_img += 1
        plt.close(fig)

    _, d_loss_print = d_train_step(Discriminator_model, batch_X, Generator_model(batch_noise)), discriminator_loss(Discriminator_model(batch_X)[1], Discriminator_model(Generator_model(batch_noise))[1])

    _, g_loss_print = g_train_step(Generator_model, Discriminator_model, batch_noise), generator_loss(Discriminator_model(Generator_model(batch_noise))[1])

    if i % 100 == 0:
        print('반복(Epoch): %d, Generator 손실함수(g_loss): %f, Discriminator 손실함수(d_loss): %f' % (i, g_loss_print, d_loss_print))